In [1]:
from astropy.io import fits
import astropy.units as u
from astropy.constants import G
from astropy.table import Table

import numpy as np

In [2]:
ic_data, header = fits.getdata("data/perseus_ic348_only.fits",header=True)
ngc_data, header = fits.getdata("data/perseus_ngc1333_only.fits",header=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/perseus_ic348_only.fits'

In [ ]:
stats = Table.read("data/deconvolution_results.csv", format = "csv")

## Mass Calculations

In [ ]:
# voxel to mass conversion
def get_mass(data):
    return np.sum(data * (1.37 * (1.6726231 * (10**(-24))) * 2.938 * (10 ** (55)))) * (5.02785 * (10**(-34)))

In [ ]:
ic_mass = get_mass(ic_data)
ngc_mass = get_mass(ngc_data)

In [ ]:
print(f"IC 348 Mass {ic_mass}")
print(f"NGC 1333 Mass {ngc_mass}")

## Dispersion Calculations

In [ ]:
def get_3ddispersion(M, rhm) :
    n = 10 #see Cottaar et al. 2015
    return np.sqrt((3*M*G)/(n * rhm)).to(u.km / u.s)
def get_dispersion(M, rhm) :
    n = 10 #see Cottaar et al. 2015
    return np.sqrt((M*G)/(n * rhm)).to(u.km / u.s)

##### IC 348

In [ ]:
ic_M_stellar = 204 * u.solMass # see Cottaar et al. 2015
ic_M_gas = ic_mass * u.solMass
ic_rhm = 0.47 * u.pc # see Cottaar et al. 2015
ic_corrected = 6.3/60 * (np.pi/180)*stats['D'][0] * u.pc
ic_rhm = stats['L_err'][0] * (np.pi/180)*stats['D'][0] * u.pc # calculated using ellipse width and distance

ic_M = ic_M_stellar.to(u.kg) + ic_M_gas.to(u.kg)
ic_R = ic_rhm.to(u.m)

In [ ]:
ic_dispersion = get_dispersion(ic_M, ic_R)
ic_3ddispersion = get_3ddispersion(ic_M, ic_R)
ic_dispersion

##### NGC 1333

In [ ]:
ngc_M_stellar = 102 * u.solMass # see Foster et al. 2015
ngc_M_gas = ngc_mass * u.solMass
# ngc_rhm = 0.97 * u. pc # see Foster et al. 2015
ngc_rhm = stats['L_err'][1] * (np.pi/180)*stats['D'][1] * u.pc # calculated using ellipse width and distance

ngc_M = ngc_M_stellar.to(u.kg) + ngc_M_gas.to(u.kg)
ngc_R = ngc_rhm.to(u.m)

In [ ]:
ngc_dispersion = get_dispersion(ngc_M, ngc_R)
ngc_3ddispersion = get_3ddispersion(ngc_M, ngc_R)


## Bounded-ness
$K = \frac{1}{2}U$

$\frac{1}{2}mv^2 = \frac{1}{4}\frac{Gm^2}{r}$

$v^2 = \frac{Gm}{2r}$

In [ ]:
print(f"NGC 1333 1D: {ngc_dispersion}")
print(f"NGC 1333 3D: {ngc_3ddispersion}")

print(f"IC 348 1D: {ic_dispersion}")
print(f"IC 348 3D: {ic_3ddispersion}")

##### IC 348

In [ ]:
G*ic_M/ic_R

In [ ]:
ic_dispersion.to(u.m/u.s)**2

In [ ]:
(1/2)*(ic_M)*(ic_dispersion.to(u.m/u.s)**2)

In [ ]:
((1/4)*G*ic_M**2)/ic_R